In [ ]:
#Credits and Thanks
#https://www.kaggle.com/datasets/midhundasl/co2-emission-of-cars-dataset
#https://github.com/krishnaik06/Pyspark-With-Python
#https://www.youtube.com/watch?v=_C8kWso4ne4&pp=ugMICgJwdBABGAE%3D

In [ ]:
!pip install pyspark

In [31]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Regression').getOrCreate()

In [32]:
data = spark.read.csv('DATA.csv',header=True,inferSchema=True)

In [33]:
data.show()

+----------+----------+------+------+---+----+
|       Car|     Model|Volume|Weight|CO2| _c5|
+----------+----------+------+------+---+----+
|    Toyota|      Aygo|  1000|   790| 99|null|
|Mitsubishi|Space Star|  1200|  1160| 95|null|
|     Skoda|    Citigo|  1000|   929| 95|null|
|      Fiat|       500|   900|   865| 90|null|
|      Mini|    Cooper|  1500|  1140|105|null|
|        VW|       Up!|  1000|   929|105|null|
|     Skoda|     Fabia|  1400|  1109| 90|null|
|  Mercedes|   A-Class|  1500|  1365| 92|null|
|      Ford|    Fiesta|  1500|  1112| 98|null|
|      Audi|        A1|  1600|  1150| 99|null|
|   Hyundai|       I20|  1100|   980| 99|null|
|    Suzuki|     Swift|  1300|   990|101|null|
|      Ford|    Fiesta|  1000|  1112| 99|null|
|     Honda|     Civic|  1600|  1252| 94|null|
|    Hundai|       I30|  1600|  1326| 97|null|
|      Opel|     Astra|  1600|  1330| 97|null|
|       BMW|         1|  1600|  1365| 99|null|
|     Mazda|         3|  2200|  1280|104|null|
|     Skoda| 

In [ ]:
data = data.drop('_c5')
data.show()

In [35]:
from pyspark.ml.feature import StringIndexer
indexer=StringIndexer(inputCols=["Car","Model"],outputCols=["Car_New","Model_New"])
data=indexer.fit(data).transform(data)
data.show()

+----------+----------+------+------+---+-------+---------+
|       Car|     Model|Volume|Weight|CO2|Car_New|Model_New|
+----------+----------+------+------+---+-------+---------+
|    Toyota|      Aygo|  1000|   790| 99|   15.0|     11.0|
|Mitsubishi|Space Star|  1200|  1160| 95|   13.0|     29.0|
|     Skoda|    Citigo|  1000|   929| 95|    2.0|     15.0|
|      Fiat|       500|   900|   865| 90|    7.0|      5.0|
|      Mini|    Cooper|  1500|  1140|105|   12.0|     17.0|
|        VW|       Up!|  1000|   929|105|   16.0|     31.0|
|     Skoda|     Fabia|  1400|  1109| 90|    2.0|     19.0|
|  Mercedes|   A-Class|  1500|  1365| 92|    1.0|      6.0|
|      Ford|    Fiesta|  1500|  1112| 98|    0.0|      0.0|
|      Audi|        A1|  1600|  1150| 99|    3.0|      7.0|
|   Hyundai|       I20|  1100|   980| 99|   10.0|     21.0|
|    Suzuki|     Swift|  1300|   990|101|   14.0|     30.0|
|      Ford|    Fiesta|  1000|  1112| 99|    0.0|      0.0|
|     Honda|     Civic|  1600|  1252| 94

In [36]:
for c in ['Car','Model']:
  data = data.drop(c)
  data = data.withColumnRenamed(f'{c}_New',c)
data.show()

+------+------+---+----+-----+
|Volume|Weight|CO2| Car|Model|
+------+------+---+----+-----+
|  1000|   790| 99|15.0| 11.0|
|  1200|  1160| 95|13.0| 29.0|
|  1000|   929| 95| 2.0| 15.0|
|   900|   865| 90| 7.0|  5.0|
|  1500|  1140|105|12.0| 17.0|
|  1000|   929|105|16.0| 31.0|
|  1400|  1109| 90| 2.0| 19.0|
|  1500|  1365| 92| 1.0|  6.0|
|  1500|  1112| 98| 0.0|  0.0|
|  1600|  1150| 99| 3.0|  7.0|
|  1100|   980| 99|10.0| 21.0|
|  1300|   990|101|14.0| 30.0|
|  1000|  1112| 99| 0.0|  0.0|
|  1600|  1252| 94| 8.0| 16.0|
|  1600|  1326| 97| 9.0| 22.0|
|  1600|  1330| 97| 5.0| 10.0|
|  1600|  1365| 99| 4.0|  1.0|
|  2200|  1280|104|11.0|  3.0|
|  1600|  1119|104| 2.0| 26.0|
|  2000|  1328|105| 0.0| 20.0|
+------+------+---+----+-----+
only showing top 20 rows



In [37]:
columns=["Volume","Weight","Car","Model"]

In [38]:
from pyspark.ml.feature import VectorAssembler
featuresAssembler=VectorAssembler(inputCols=columns,outputCol="Independent Features")
data = featuresAssembler.transform(data)

In [39]:
data.show()

+------+------+---+----+-----+--------------------+
|Volume|Weight|CO2| Car|Model|Independent Features|
+------+------+---+----+-----+--------------------+
|  1000|   790| 99|15.0| 11.0|[1000.0,790.0,15....|
|  1200|  1160| 95|13.0| 29.0|[1200.0,1160.0,13...|
|  1000|   929| 95| 2.0| 15.0|[1000.0,929.0,2.0...|
|   900|   865| 90| 7.0|  5.0|[900.0,865.0,7.0,...|
|  1500|  1140|105|12.0| 17.0|[1500.0,1140.0,12...|
|  1000|   929|105|16.0| 31.0|[1000.0,929.0,16....|
|  1400|  1109| 90| 2.0| 19.0|[1400.0,1109.0,2....|
|  1500|  1365| 92| 1.0|  6.0|[1500.0,1365.0,1....|
|  1500|  1112| 98| 0.0|  0.0|[1500.0,1112.0,0....|
|  1600|  1150| 99| 3.0|  7.0|[1600.0,1150.0,3....|
|  1100|   980| 99|10.0| 21.0|[1100.0,980.0,10....|
|  1300|   990|101|14.0| 30.0|[1300.0,990.0,14....|
|  1000|  1112| 99| 0.0|  0.0|[1000.0,1112.0,0....|
|  1600|  1252| 94| 8.0| 16.0|[1600.0,1252.0,8....|
|  1600|  1326| 97| 9.0| 22.0|[1600.0,1326.0,9....|
|  1600|  1330| 97| 5.0| 10.0|[1600.0,1330.0,5....|
|  1600|  13

In [40]:
from pyspark.ml.regression import LinearRegression
train_data,test_data=data.randomSplit([0.7,0.3])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='CO2')
regressor=regressor.fit(train_data)

In [41]:
for i, c in enumerate(regressor.coefficients):
  print(f'{columns[i]}*{c:.3f}',end=" + ")
print(f'{regressor.intercept:.3f}')

Volume*0.012 + Weight*0.008 + Car*0.394 + Model*0.058 + 68.307


In [44]:
def calc(*args):
  sum = 0
  for i,c in enumerate(regressor.coefficients):
    sum += args[i] * c
  sum += regressor.intercept
  return sum

In [42]:
pred=regressor.evaluate(test_data)

In [43]:
pred.predictions.show()

+------+------+---+----+-----+--------------------+------------------+
|Volume|Weight|CO2| Car|Model|Independent Features|        prediction|
+------+------+---+----+-----+--------------------+------------------+
|   900|   865| 90| 7.0|  5.0|[900.0,865.0,7.0,...| 89.42421951663486|
|  1000|   929|105|16.0| 31.0|[1000.0,929.0,16....| 96.23449811122855|
|  1000|  1112| 99| 0.0|  0.0|[1000.0,1112.0,0....| 89.57561599571888|
|  1100|   980| 99|10.0| 21.0|[1100.0,980.0,10....| 94.93899720358334|
|  1500|  1112| 98| 0.0|  0.0|[1500.0,1112.0,0....| 95.80967150060057|
|  1500|  1140|105|12.0| 17.0|[1500.0,1140.0,12...|101.74913315015507|
|  1600|  1326| 97| 9.0| 22.0|[1600.0,1326.0,9....|103.57506804227765|
|  1600|  1365| 99| 4.0|  1.0|[1600.0,1365.0,4....| 100.6940517931546|
|  2000|  1490|104| 3.0|  8.0|[2000.0,1490.0,3....|106.68229156384723|
|  2000|  1725|114| 3.0|  9.0|[2000.0,1725.0,3....| 108.6001420410588|
|  2200|  1280|104|11.0|  3.0|[2200.0,1280.0,11...| 110.3784262820143|
+-----

In [45]:
calc(900,865,7,5)

89.42421951663486

In [46]:
calc(1000,929,16,31)

96.23449811122855

In [47]:
pred.meanAbsoluteError,pred.meanSquaredError

(4.63614183293761, 29.180071399474958)